# 패키지 다운로드

In [ ]:
# pip install xarray

# 패키지 로딩

In [24]:
import xarray as xr

import pandas as pd
import numpy as np

import tensorflow as tf

import warnings

warnings.filterwarnings("ignore")

# 종속변수(SCTR) 전처리

## nc -> csv 변환

In [ ]:
ds = xr.open_dataset('./data/EN.4.2.2_D20_SCTR_f.nc')

df = ds.D20_SCTR.to_dataframe()

df.reset_index(inplace=True)

df.to_csv('./output/SCTR.csv', index=False)

ds.close()

# 종속변수 불러오기

In [3]:
sctr = pd.read_csv("./output/SCTR.csv")

# 날짜 형식 통일
sctr["TIME1"] = pd.to_datetime(sctr["TIME1"])
sctr["TIME1"] = sctr["TIME1"].dt.to_period('M').dt.to_timestamp()

# 날짜 컬럼 이름 변경
sctr = sctr.rename(columns={"TIME1":"Date"})

sctr

,Date,D20_SCTR
0,1991-01-01,54.960640
1,1991-02-01,59.899442
2,1991-03-01,70.225124
3,1991-04-01,67.375125
4,1991-05-01,63.783066
...,...,...
395,2023-12-01,124.376897
396,2024-01-01,117.300079
397,2024-02-01,121.968669
398,2024-03-01,118.355688


# 독립변수 전처리

## IOD
- WTIO  : 동경 50도 ~ 70도, 남위 10도 ~ 북위 10도에서 SST 편차의 평균 
- SETIO : 동경 90도 ~ 110도, 남위 10도 ~ 적도선에서 SST 편차의 평균
- DMI  = WTIO - SETIO

### txt -> csv 변환

In [ ]:
IOD = pd.read_csv("./data/IOD.txt", sep="\t", encoding="cp949")

# 날짜 형식 통일
IOD['Date'] = pd.to_datetime(IOD['Year'].astype(str) + '-' + IOD['Month'].astype(str) + '-01')
IOD = IOD.drop(["Year", "Month"], axis = 1)
IOD = IOD[["Date", "WTIO", "SETIO"]]

# DMI(Dipole Mode Index) 컬럼 생성
IOD["DMI"] = IOD["WTIO"] - IOD["SETIO"]

IOD.to_csv("./output/IOD.csv", encoding="utf-8-sig", index=False)

In [ ]:
IOD = pd.read_csv("./output/IOD.csv")

IOD

## ONI

### txt -> csv 변환

In [ ]:
ONI = pd.read_csv("./data/ONI.txt", sep="\t", encoding="cp949")

# 날짜 형식 통일
ONI['Date'] = pd.to_datetime(ONI['Date'].astype(str) + '-01')

ONI.to_csv("./output/ONI.csv", encoding="utf-8-sig", index=False)

In [ ]:
ONI = pd.read_csv("./output/ONI.csv")

ONI

## OISST
- NINO 3.4(북위 5도 ~ 남위 5도, 서경 170도 ~ 120도)의 평균 해수면 온도 & 편차

### txt -> csv 변환

In [ ]:
OISST = pd.read_csv("./data/OISST.txt", sep="\t", encoding="cp949")

OISST['Date'] = pd.to_datetime(OISST['Date'])

OISST.to_csv("./output/OISST.csv", encoding="utf-8-sig", index=False)

In [ ]:
OISST = pd.read_csv("./output/OISST.csv")

OISST

## SOI
- 남방진동지수, Darwin - Tahiti 사이의 표준화된 해면기압 차이

### txt -> csv 변환

In [ ]:
SOI = pd.read_csv("./data/SOI.txt", sep="\t", encoding="cp949")

# 날짜 형식 통일
SOI['Date'] = pd.to_datetime(SOI['Date'].astype(str) + '-01')

SOI.to_csv("./output/SOI.csv", encoding="utf-8-sig", index=False)

In [ ]:
SOI = pd.read_csv("./output/SOI.csv")

SOI

## 독립변수 합치기

In [ ]:
df = pd.merge(IOD, ONI, on="Date")
df = pd.merge(df, OISST, on="Date")
df = pd.merge(df, SOI, on="Date")

df.to_csv("./output/IND.csv", encoding="utf-8-sig", index=False)

# 독립변수 불러오기 & 종속변수 합치기

In [4]:
ind = pd.read_csv("./output/IND.csv")

ind["Date"] = pd.to_datetime(ind["Date"])

data = pd.merge(ind, sctr, on="Date").drop("Date", axis = 1)

# OISST 빼기(NINO3.4, ANOM)

In [5]:
data = data.drop(["NINO3.4", "ANOM"], axis = 1)

data

,WTIO,SETIO,DMI,ONI,SOI,D20_SCTR
0,0.18,0.12,0.06,0.41,0.6,54.960640
1,0.05,-0.04,0.09,0.26,0.3,59.899442
2,-0.04,0.02,-0.06,0.22,-0.7,70.225124
3,0.31,-0.45,0.76,0.26,-0.6,67.375125
4,0.28,-0.62,0.90,0.45,-1.0,63.783066
...,...,...,...,...,...,...
395,1.14,-0.07,1.21,1.95,-0.2,124.376897
396,1.18,0.36,0.82,1.79,0.5,117.300079
397,1.22,0.92,0.30,1.49,-1.4,121.968669
398,0.95,0.41,0.54,1.14,0.4,118.355688


# 중간정리
- IOD(WTIO, SETIO, DMI)는 WTIO, SETIO / DMI로 나눠서 모델 구성
- SOI는 이미 표준화되어 있으므로 표준화 O, X 나눠서 모델 구성

# 함수 로딩

In [6]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size

    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i + target_size])
            
    return np.array(data), np.array(labels)

In [7]:
def plot_train_history(history, function):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))
    
    plt.rcParams['font.family'] ='Malgun Gothic'
    plt.figure(figsize=(12, 8))
    
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    
    plt.xlabel("Epoch")
    plt.ylabel(function)
    
    plt.title("모델 손실함수 시각화")
    plt.legend()
    plt.grid()

    plt.show()

# Case1) WTIO, SETIO & SOI 표준화

## 데이터 전처리

In [21]:
case1_df = data[['WTIO', 'SETIO', 'ONI', 'SOI', 'D20_SCTR']]

case1_dts = case1_df.values

case1_mean = case1_dts.mean(axis=0)
case1_std = case1_dts.std(axis=0)
case1_dts = (case1_dts-case1_mean)/case1_std

In [58]:
case1_df

,WTIO,SETIO,ONI,SOI,D20_SCTR
0,0.18,0.12,0.41,0.6,54.960640
1,0.05,-0.04,0.26,0.3,59.899442
2,-0.04,0.02,0.22,-0.7,70.225124
3,0.31,-0.45,0.26,-0.6,67.375125
4,0.28,-0.62,0.45,-1.0,63.783066
...,...,...,...,...,...
395,1.14,-0.07,1.95,-0.2,124.376897
396,1.18,0.36,1.79,0.5,117.300079
397,1.22,0.92,1.49,-1.4,121.968669
398,0.95,0.41,1.14,0.4,118.355688


In [44]:
len(case1_dts)

400

## 하이퍼 파라매터 세팅

In [73]:
# 학습(0.7) - 평가(0.2) - 검증(0.1) 데이터 분리
TRAIN_SPLIT = int(len(case1_dts) * 0.7)
TEST_SPLIT = int(len(case1_dts) * 0.2)

BATCH_SIZE = 32
EPOCHS = 120

# 12개월 학습 - 12개월 예측
past_history = 12
future_target = 12
STEP = 1

tf.random.set_seed(13)

In [61]:
TRAIN_SPLIT

280

In [59]:
TEST_SPLIT

80

In [60]:
x_train_multi, y_train_multi = multivariate_data(case1_dts, case1_dts[:, -1], 0, TRAIN_SPLIT, past_history, future_target, STEP)

x_val_multi, y_val_multi = multivariate_data(case1_dts, case1_dts[:, -1], TRAIN_SPLIT, TRAIN_SPLIT + TEST_SPLIT, past_history, future_target, STEP)

x_test_multi, y_test_multi = multivariate_data(case1_dts, case1_dts[:, -1], TRAIN_SPLIT + TEST_SPLIT, None, past_history, future_target, STEP)

print(x_train_multi.shape)
print('Single window of past history : {}'.format(x_train_multi[0].shape))
print('Target time to predict : {}'.format(y_train_multi[0].shape))

print(x_val_multi.shape)

print(x_test_multi.shape)

(268, 12, 5)
Single window of past history : (12, 5)
Target time to predict : (12,)
(68, 12, 5)
(16, 12, 5)


In [66]:
# 전체 데이터 길이
total_length = len(case1_dts)

# 학습(0.7) - 검증(0.2) - 테스트(0.1) 데이터 분리
TRAIN_SPLIT = int(total_length * 0.7)
VAL_SPLIT = int(total_length * 0.2)
TEST_SPLIT = total_length - TRAIN_SPLIT - VAL_SPLIT

# 12개월 학습 - 12개월 예측
past_history = 12
future_target = 12
STEP = 1

# 학습 데이터
x_train_multi, y_train_multi = multivariate_data(case1_dts, case1_dts[:, -1], 0, TRAIN_SPLIT - future_target, past_history, future_target, STEP)

# 검증 데이터
x_val_multi, y_val_multi = multivariate_data(case1_dts, case1_dts[:, -1], TRAIN_SPLIT, TRAIN_SPLIT + VAL_SPLIT - future_target, past_history, future_target, STEP)

# 테스트 데이터
x_test_multi, y_test_multi = multivariate_data(case1_dts, case1_dts[:, -1], TRAIN_SPLIT + VAL_SPLIT, None, past_history, future_target, STEP)

print(f"학습 데이터 형태: {x_train_multi.shape}")
print(f"검증 데이터 형태: {x_val_multi.shape}")
print(f"테스트 데이터 형태: {x_test_multi.shape}")

print('과거 데이터 윈도우 형태: {}'.format(x_train_multi[0].shape))
print('예측 목표 기간 형태: {}'.format(y_train_multi[0].shape))

학습 데이터 형태: (256, 12, 5)
검증 데이터 형태: (56, 12, 5)
테스트 데이터 형태: (16, 12, 5)
과거 데이터 윈도우 형태: (12, 5)
예측 목표 기간 형태: (12,)


In [75]:
BUFFER_SIZE = 100

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))

train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

In [76]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# 학습률 감소
# def lr_scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# lr_scheduler = LearningRateScheduler(lr_scheduler, verbose=1)    
    
initial_lr = 0.001
optimizer = RMSprop(learning_rate=initial_lr, clipvalue=1.0)

# 학습률 재시작
lr_restart = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

multi_step_model = Sequential()

multi_step_model.add(LSTM(32, return_sequences=True, input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(Dropout(0.2))
multi_step_model.add(LSTM(16, activation='relu'))
multi_step_model.add(Dropout(0.2))
multi_step_model.add(Dense(12))

multi_step_model.compile(optimizer=optimizer, loss='mae')

multi_step_history = multi_step_model.fit(
    train_data_multi, 
    epochs = EPOCHS, 
    steps_per_epoch = x_train_multi.shape[0] // BATCH_SIZE, 
    validation_data = val_data_multi, 
    validation_steps = x_val_multi.shape[0] // BATCH_SIZE,
    callbacks=[
#         lr_scheduler, 
        lr_restart, 
        early_stop
    ])

plot_train_history(multi_step_history, "Case1")

Epoch 1/120
8/8 [==============================] - 5s 107ms/step - loss: 0.7113 - val_loss: 0.7701 - lr: 0.0010
Epoch 2/120
8/8 [==============================] - 0s 11ms/step - loss: 0.7065 - val_loss: 0.7674 - lr: 0.0010
Epoch 3/120
8/8 [==============================] - 0s 11ms/step - loss: 0.7023 - val_loss: 0.7650 - lr: 0.0010
Epoch 4/120
8/8 [==============================] - 0s 12ms/step - loss: 0.6986 - val_loss: 0.7648 - lr: 0.0010
Epoch 5/120
8/8 [==============================] - 0s 11ms/step - loss: 0.6936 - val_loss: 0.7662 - lr: 0.0010
Epoch 6/120
8/8 [==============================] - 0s 11ms/step - loss: 0.6936 - val_loss: 0.7673 - lr: 0.0010
Epoch 7/120
8/8 [==============================] - 0s 12ms/step - loss: 0.6873 - val_loss: 0.7698 - lr: 0.0010
Epoch 8/120
8/8 [==============================] - 0s 10ms/step - loss: 0.6834 - val_loss: 0.7680 - lr: 0.0010
Epoch 9/120
7/8 [=========================>....] - ETA: 0s - loss: 0.6832
Epoch 9: ReduceLROnPlateau reducing l

NameError: name 'plt' is not defined